In [2]:
import tkinter as tk
from tkinter import ttk
from tkinter.scrolledtext import ScrolledText
import pandas as pd
import re

pd.options.mode.chained_assignment = None

# -------------------- 전처리 함수 --------------------
def normalize_station(name: str) -> str:
    name = str(name)
    name = re.sub(r"\(.*?\)", "", name).strip()
    return name if name.endswith("역") else name + "역"

# -------------------- 핵심 계산 함수 --------------------
def calculate_recommendations():
    yearly_files = {
        2020: "서울교통공사_역별 시간대별 승하차인원(20.1~20.12).csv",
        2021: "서울교통공사_역별 시간대별 승하차인원(21.1~21.12).csv",
        2022: "서울교통공사_역별 시간대별 승하차인원(22.1~22.12).csv",
        2023: "서울교통공사_역별 시간대별 승하차인원(23.1~23.12).csv",
        2024: "서울교통공사_역별 시간대별 승하차인원(24.1~24.12).csv",
    }

    daily_all = []
    for year, path in yearly_files.items():
        df = pd.read_csv(path, encoding="cp949", low_memory=False)
        a_col = [c for c in df.columns if "구분" in c][0]
        date_col = [c for c in df.columns if "사용일자" in c or "일자" in c]
        days = 366 if year % 4 == 0 else 365
        if date_col:
            df[date_col[0]] = pd.to_datetime(df[date_col[0]], errors="coerce")
            days = df[date_col[0]].dt.normalize().nunique()

        ride_df = df[df[a_col] == "승차"].copy()
        exit_df = df[df[a_col] == "하차"].copy()
        time_cols = [c for c in df.columns if ("~" in c or "시" in c) and "승강" not in c]
        ride_df[time_cols] = ride_df[time_cols].apply(pd.to_numeric, errors="coerce")
        exit_df[time_cols] = exit_df[time_cols].apply(pd.to_numeric, errors="coerce")
        total = ride_df.groupby("역명")[time_cols].sum().sum(axis=1) + \
                exit_df.groupby("역명")[time_cols].sum().sum(axis=1)
        daily = (total / days).reset_index(name="일평균이용객")
        daily["정규역명"] = daily["역명"].apply(normalize_station)
        daily_all.append(daily[["정규역명", "일평균이용객"]])

    daily_df = pd.concat(daily_all, ignore_index=True)
    avg_pop = daily_df.groupby("정규역명")["일평균이용객"].mean().reset_index(name="평균유동인구")

    accident_df = pd.read_csv("서울교통공사_최근 5년 지하철 사고 현황_20250310.csv", encoding="cp949")
    accident_df["정규역명"] = accident_df["발생역"].apply(normalize_station)

    ext_df = pd.read_csv("서울교통공사_소화기설치현황_20250310.csv", encoding="cp949")
    ext_df["정규역명"] = ext_df["역명"].apply(normalize_station)
    ext_qty_col = [c for c in ext_df.columns if "보유" in c or "수" in c][0]
    ext_cnt = ext_df.groupby("정규역명")[ext_qty_col].sum().reset_index(name="소화기수")

    mask_df = pd.read_csv("서울교통공사_역별 화재용 대피마스크 현황_20250310.csv", encoding="cp949")
    mask_df["정규역명"] = mask_df["역명"].apply(normalize_station)
    mask_qty_col = [c for c in mask_df.columns if "보유" in c or "수" in c][0]
    mask_cnt = mask_df.groupby("정규역명")[mask_qty_col].sum().reset_index(name="대피마스크수")

    arch_df = pd.read_csv("서울교통공사_역사건축정보_20250310.csv", encoding="cp949")
    arch_df["정규역명"] = arch_df["역명"].apply(normalize_station)
    arch_area = arch_df.groupby("정규역명")["면적"].sum().reset_index(name="역면적")

    accident_weights = {
        "출입문관련": {"weight": 1.3, "action": "출입문 센서 점검 및 승하차 유도선 정비"},
        "역구내 사고": {"weight": 1.2, "action": "계단 미끄럼 방지 패드 설치, 안내판 보강"},
        "열차내 사고": {"weight": 1.1, "action": "차내 안내방송 개선 및 손잡이 정비"},
        "발빠짐": {"weight": 1.4, "action": "승강장 틈새 방지 설비 추가"},
        "승강설비관련": {"weight": 1.5, "action": "엘리베이터/에스컬레이터 정기 점검 강화"},
        "기타": {"weight": 0.8, "action": "CCTV 및 비상벨 설치 확대"},
    }

    summary = accident_df.groupby(["정규역명", "사고유형"]).size().unstack(fill_value=0)
    summary["사고건수"] = summary.sum(axis=1)
    for t, meta in accident_weights.items():
        summary[f"{t}_점수"] = summary.get(t, 0) * meta["weight"]
    summary["사고유형점수합"] = summary[[f"{t}_점수" for t in accident_weights]].sum(axis=1)

    summary = summary.reset_index()
    summary = summary.merge(avg_pop, on="정규역명", how="left")
    summary = summary.merge(arch_area, on="정규역명", how="left")
    summary = summary.merge(ext_cnt, on="정규역명", how="left")
    summary = summary.merge(mask_cnt, on="정규역명", how="left")
    summary = summary.set_index("정규역명")

    summary["유동인구밀도"] = summary["평균유동인구"] / summary["역면적"]
    summary["소화기밀도"] = summary["소화기수"] / summary["역면적"]
    summary["마스크인구비"] = summary["대피마스크수"] / summary["평균유동인구"]

    ext_density_med = summary["소화기밀도"].median()
    mask_ratio_med = summary["마스크인구비"].median()

    global type_ratio
    type_ratio = summary[[k for k in accident_weights]].div(summary["사고건수"], axis=0)

    def recommend(row):
        actions, reasons = set(), []
        for t in type_ratio.loc[row.name].sort_values(ascending=False).head(2).index:
            actions.add(accident_weights[t]["action"])
            reasons.append(f"{t} 사고 비율이 높음")
        if pd.notna(row["유동인구밀도"]) and row["유동인구밀도"] > summary["유동인구밀도"].quantile(0.8):
            actions.add("역 내 유동 동선 재배치 또는 확장")
            reasons.append("역 크기에 비해 유동인구가 많음")
        if pd.notna(row["소화기밀도"]) and row["소화기밀도"] < ext_density_med:
            actions.add("소화기 추가 설치")
            reasons.append("소화기 밀도가 낮음")
        if pd.notna(row["마스크인구비"]) and row["마스크인구비"] < mask_ratio_med:
            actions.add("대피마스크 비치 확대")
            reasons.append("유동인구 대비 대피마스크 부족")
        return " + ".join(actions), " / ".join(reasons)

    recommendations = summary.apply(lambda r: recommend(r), axis=1, result_type="expand")
    recommendations.columns = ["추천 보수 조치", "추천 이유"]
    top20_accidents = summary.sort_values("사고건수", ascending=False).head(20)[["사고건수"]]
    return recommendations, top20_accidents, sorted(recommendations.index.tolist()), summary

# -------------------- 데이터 계산 --------------------
recommend_df, top20_df, station_names, summary = calculate_recommendations()

# -------------------- GUI 함수 --------------------
def update_output(text):
    output_text.config(state="normal")
    output_text.delete("1.0", tk.END)
    output_text.insert(tk.END, text)
    output_text.config(state="disabled")

def search_station():
    query = entry.get().strip()
    if not query.endswith("역"):
        query += "역"

    if query in recommend_df.index:
        result = recommend_df.loc[query]
        row = summary.loc[query]

        ratio_row = type_ratio.loc[query]
        ratio_str = "\n".join(f"{col}: {val:.1%}" for col, val in ratio_row.items() if val > 0)
        acc_cnt   = f"{int(row['사고건수'])}" if pd.notna(row['사고건수']) else "정보 없음"

        pop = f"{row['평균유동인구']:.0f}" if pd.notna(row['평균유동인구']) else "정보 없음"
        area = f"{row['역면적']:.2f}" if pd.notna(row['역면적']) else "정보 없음"
        density = f"{row['유동인구밀도']:.3f}" if pd.notna(row['유동인구밀도']) else "정보 없음"

        rank_acc  = summary['사고건수'].rank(ascending=False, method='min')   # ➊

        ext_cnt = f"{int(row['소화기수'])}" if pd.notna(row['소화기수']) else "정보 없음"
        ext_density = f"{row['소화기밀도']:.5f}" if pd.notna(row['소화기밀도']) else "정보 없음"
        mask_cnt = f"{int(row['대피마스크수'])}" if pd.notna(row['대피마스크수']) else "정보 없음"
        mask_ratio = f"{row['마스크인구비']:.5f}" if pd.notna(row['마스크인구비']) else "정보 없음"

        rank_pop = summary['유동인구밀도'].rank(ascending=False, method='min')
        rank_ext = summary['소화기밀도'].rank(ascending=False, method='min')
        rank_mask = summary['마스크인구비'].rank(ascending=False, method='min')

        pop_rank_str  = f"{int(rank_pop[query])}위 / {int(rank_pop.count())}개 역"  if pd.notna(row['유동인구밀도']) else "정보 없음"
        ext_rank_str  = f"{int(rank_ext[query])}위 / {int(rank_ext.count())}개 역"  if pd.notna(row['소화기밀도'])  else "정보 없음"
        mask_rank_str = f"{int(rank_mask[query])}위 / {int(rank_mask.count())}개 역" if pd.notna(row['마스크인구비']) else "정보 없음"
        acc_rank  = f"{int(rank_acc[query])}위 / {int(rank_acc.count())}개 역" \
           if pd.notna(row['사고건수']) else "정보 없음"     
        result_text = (
            f"📍 역명: {query}\n\n"
            f"✅ 추천 보수 조치:\n{result['추천 보수 조치']}\n\n"
            f"📝 이유:\n{result['추천 이유']}\n\n"
            f"💥 사고 건수(최근 5년): {acc_cnt}건  (순위: {acc_rank})\n"       
            f"📊 사고 유형 비율:\n{ratio_str}\n\n"
            f"🏙 평균 유동인구: {pop}\n"
            f"📏 역 면적(㎡): {area}\n"
            f"👥 유동인구 밀도: {density} 명/㎡\n"
            f"🏅 유동인구 밀도 순위: {pop_rank_str}\n\n"
            f"🧯 소화기 수: {ext_cnt} (밀도 {ext_density} 개/㎡)\n"
            f"   👉 소화기 밀도 순위: {ext_rank_str}\n\n"
            f"😷 대피마스크 수: {mask_cnt} (인구비 {mask_ratio} 개/명)\n"
            f"   👉 마스크 인구비 순위: {mask_rank_str}"
        )
    else:
        result_text = "해당 역을 찾을 수 없습니다."

    update_output(result_text)



metrics = {
    "유동인구 밀도": "유동인구밀도",
    "소화기 밀도": "소화기밀도",
    "마스크 인구비": "마스크인구비",
    "사고 건수":   "사고건수",
}

def search_rank():
    metric_disp = metric_var.get()
    metric_col  = metrics[metric_disp]

    try:
        start = int(rank_from_entry.get()) if rank_from_entry.get() else 1
        end   = int(rank_to_entry.get())   if rank_to_entry.get()   else 20
    except ValueError:
        update_output("순위 입력은 숫자여야 합니다.")
        return
    if start < 1 or end < start:
        update_output("순위 범위를 올바르게 입력하세요.")
        return

    ranks = summary[metric_col].rank(ascending=False, method='min')
    ranked = ranks.to_frame("순위").join(summary[[metric_col]])
    subset = ranked[(ranked["순위"] >= start) & (ranked["순위"] <= end)].sort_values("순위")

    if subset.empty:
        update_output("해당 구간에 역이 없습니다.")
        return

    text = f"📈 {metric_disp} {start}위 ~ {end}위:\n\n"
    for idx, row in subset.iterrows():
        val = f"{row[metric_col]:.5f}" if metric_col != "사고건수" else f"{int(row[metric_col])}"
        text += f"{int(row['순위'])}위  {idx}: {val}\n"
    update_output(text)

# -------------------- Tkinter GUI --------------------
root = tk.Tk()
root.title("지하철역 보수 추천 시스템")
root.geometry("750x750")
root.option_add("*Font", "Helvetica 11")

style = ttk.Style(root)
style.configure("TButton", padding=6)   

# 검색 섹션
search_frame = ttk.Frame(root, padding=(10, 15))
search_frame.pack(fill="x")

ttk.Label(search_frame, text="지하철역 검색:", font=("Helvetica", 12, "bold")).pack(anchor="w")
entry = ttk.Combobox(search_frame, font=("Helvetica", 14), width=30, values=station_names)
entry.pack(pady=4)

button_frame = ttk.Frame(search_frame)
button_frame.pack(pady=4, fill="x")
ttk.Button(button_frame, text="검색", command=search_station).pack(side="left", padx=2)

# ------------------------ 가능한 검색 범위 ------------------------
rankable_info = {
    "유동인구 밀도": 229,
    "소화기 밀도": 225,
    "마스크 인구비": 211,
    "사고 건수": 229
}

desc_text = (
    "① 지표를 선택하고 ② 시작·끝 순위를 입력한 뒤 ③ [순위 검색]을 누르세요.\n\n"
    "📌 [검색 가능 순위 범위]\n" +
    "\n".join(f"· {k}: 1위 ~ {v}위" for k, v in rankable_info.items())
)
# -----------------------------------------------------------------

# ---------- 순위 범위 검색 위젯 ----------
rank_frame = ttk.Labelframe(root, text="순위 범위 검색", padding=(10, 10))
rank_frame.pack(fill="x", padx=10, pady=8)

desc_lbl = ttk.Label(rank_frame, text=desc_text, wraplength=700, foreground="#444", justify="left")

desc_lbl.pack(anchor="w", pady=2)

metric_var = tk.StringVar(value="유동인구 밀도")
metric_menu = ttk.Combobox(rank_frame, textvariable=metric_var,
                           values=list(metrics.keys()), state="readonly", width=18)
metric_menu.pack(pady=2, anchor="w")

range_inner = ttk.Frame(rank_frame)
range_inner.pack(pady=2, anchor="w")
ttk.Label(range_inner, text="시작:").pack(side="left", padx=(0,2))
rank_from_entry = ttk.Entry(range_inner, width=6)
rank_from_entry.pack(side="left")
ttk.Label(range_inner, text="끝:").pack(side="left", padx=(8,2))
rank_to_entry = ttk.Entry(range_inner, width=6)
rank_to_entry.pack(side="left", padx=(0,6))

ttk.Button(rank_frame, text="순위 검색", command=search_rank).pack(pady=6)

# ---------------- 결과 출력창 ----------------
output_frame = ttk.Frame(root, padding=(10, 5))
output_frame.pack(fill="both", expand=True, padx=10, pady=(0,10))

output_text = ScrolledText(output_frame, wrap="word", font=("Courier New", 11),
                           height=22, state="disabled")
output_text.pack(fill="both", expand=True)

root.mainloop()
